In [1]:
#Q2.1
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.0.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0 pyspark-shell'
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, decode, expr

conf = SparkConf().setAppName("Assignment-2B-Task2_spark_streaming ").setMaster("local[2]").set("spark.sql.session.timeZone", "UTC")
spark = SparkSession.builder.config(conf = conf).getOrCreate()

In [2]:
#importing necessary libraries
from pyspark.sql.functions import explode
from pyspark.sql.functions import *
from pyspark.sql import functions as F
from pyspark.sql.types import *
from datetime import datetime

In [3]:
##Q2.2

#injesting bureau data
bureau_stream = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "127.0.0.1:9092") \
  .option("subscribe", 'Bureau_data') \
  .option("startingOffsets", "latest")\
  .load()


In [4]:
#injesting customer data
topic = 'Customer_Data'
customer_stream = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "127.0.0.1:9092") \
  .option("subscribe",topic) \
  .option("startingOffsets", "latest")\
  .load()

In [5]:
#Assume all data is string
df_bureau = bureau_stream.selectExpr( "CAST(value AS STRING)")

In [6]:
df_customer = customer_stream.selectExpr("CAST(value AS STRING)")

In [7]:
##Q2.3
#Defining the Schema for Customer dataframe
from pyspark.sql.types import StructField,IntegerType, StructType,StringType, DoubleType
schema = ArrayType(StructType([
       StructField('ID',StringType(),True),
       StructField('Frequency',StringType(),True),
       StructField('InstlmentMode',StringType(),True),
       StructField('LoanStatus',StringType(),True),
       StructField('PaymentMode',StringType(),True),
       StructField('BranchID',StringType(),True),
       StructField('Area',StringType(),True) ,
       StructField('Tenure',StringType(),True), 
       StructField('AssetCost',StringType(),True) ,
       StructField('AmountFinance',StringType(),True) ,
       StructField('DisbursalAmount',StringType(),True), 
       StructField('EMI',StringType(),True), 
       StructField('DisbursalDate',StringType(),True) ,
       StructField('MaturityDAte',StringType(),True), 
       StructField('AuthDate',StringType(),True), 
       StructField('AssetID',StringType(),True),
       StructField('ManufacturerID',StringType(),True),
       StructField('SupplierID',StringType(),True),
       StructField('LTV',StringType(),True), 
       StructField('SEX',StringType(),True), 
       StructField('AGE',StringType(),True), 
       StructField('MonthlyIncome',StringType(),True), 
       StructField('City',StringType(),True),
       StructField('State',StringType(),True),
       StructField('ZiPCODE',StringType(),True),
       StructField('Top-up Month',StringType(),True),
       StructField('ts',StringType(),True)
       ]))

In [8]:
#Transforming the customer data
df_customer = df_customer.select(F.from_json(F.col("value").cast("string"), schema).alias('parsed_value'))
df_customer = df_customer.select(F.explode(F.col("parsed_value")).alias('unnested_value'))

In [9]:
df_customer = df_customer.select(
                    F.col("unnested_value.ID").alias("ID"),
                    F.col("unnested_value.Frequency").alias("Frequency"),
                    F.col("unnested_value.InstlmentMode").alias("InstlmentMode"),
                    F.col("unnested_value.LoanStatus").alias("LoanStatus"),
                    F.col("unnested_value.PaymentMode").alias("PaymentMode"),
                    F.col("unnested_value.BranchID").alias("BranchID"),
                    F.col("unnested_value.Area").alias("Area"),
                    F.col("unnested_value.Tenure").alias("Tenure"),
                    F.col("unnested_value.InstlmentMode").alias("InstlmentMode"),
                    F.col("unnested_value.AssetCost").alias("AssetCost"),
                    F.col("unnested_value.AmountFinance").alias("AmountFinance"),
                    F.col("unnested_value.DisbursalAmount").alias("DisbursalAmount"),
                    F.col("unnested_value.EMI").alias("EMI"),
                    F.col("unnested_value.DisbursalDate").alias("DisbursalDate"),
                    F.col("unnested_value.MaturityDAte").alias("MaturityDAte"),
                    F.col("unnested_value.AuthDate").alias("AuthDatee"),
                    F.col("unnested_value.AssetID").alias("AssetID"),
                    F.col("unnested_value.ManufacturerID").alias("ManufacturerID"),
                    F.col("unnested_value.SupplierID").alias("SupplierID"),
                    F.col("unnested_value.LTV").alias("LTV"),
                    F.col("unnested_value.SEX").alias("SEX"),
                    F.col("unnested_value.AGE").alias("AGE"),
                    F.col("unnested_value.MonthlyIncome").alias("MonthlyIncome"),
                    F.col("unnested_value.City").alias("City"),
                    F.col("unnested_value.State").alias("State"),
                    F.col("unnested_value.ZiPCODE").alias("ZiPCODE"),
                    F.col("unnested_value.Top-up Month").alias("Top-up Month"),
                    F.col("unnested_value.ts").alias("ts")                    
                )

In [10]:
#Casting the non-string columns to appropriate datatypes
df_customer = df_customer.withColumn("ID", col("ID").cast(IntegerType()))


df_customer = df_customer.withColumn("Tenure", col("Tenure").cast(IntegerType()))


df_customer = df_customer.withColumn("AssetCost", col("AssetCost").cast(IntegerType()))
df_customer = df_customer.withColumn("AmountFinance", col("AmountFinance").cast(DoubleType()))
df_customer = df_customer.withColumn("DisbursalAmount", col("DisbursalAmount").cast(DoubleType()))
df_customer = df_customer.withColumn("EMI", col("EMI").cast(DoubleType()))

df_customer = df_customer.withColumn("LTV", col("LTV").cast(DoubleType()))
df_customer = df_customer.withColumn("AGE", col("AGE").cast(IntegerType()))
df_customer = df_customer.withColumn("MonthlyIncome", col("MonthlyIncome").cast(DoubleType()))

df_customer = df_customer.withColumn("ZiPCODE", col("ZiPCODE").cast(IntegerType()))
df_customer = df_customer.withColumn("ts", col('ts').cast(IntegerType()))
df_customer = df_customer.withColumn("ts", from_unixtime('ts').alias('ts'))
df_customer = df_customer.withColumn("ts",col("ts").cast(TimestampType()))

In [11]:
#Defining the Schema for Customer dataframe
from pyspark.sql.types import StructField,IntegerType, StructType,StringType, DoubleType,BooleanType,TimestampType
schema_b = ArrayType(StructType([
       StructField('ID',StringType(),True),
       StructField('SELF-INDICATOR',StringType(),True),
       StructField('MATCH-TYPE',StringType(),True),
       StructField('ACCT-TYPE',StringType(),True),
       StructField('CONTRIBUTOR-TYPE',StringType(),True),
       StructField('DATE-REPORTED',StringType(),True),
       StructField('OWNERSHIP-IND',StringType(),True) ,
       StructField('ACCOUNT-STATUS',StringType(),True), 
       StructField('DISBURSED-DT',StringType(),True) ,
       StructField('CLOSE-DT',StringType(),True) ,
       StructField('LAST-PAYMENT-DATE',StringType(),True), 
       StructField('CREDIT-LIMIT/SANC AMT',StringType(),True), 
       StructField('DISBURSED-AMT/HIGH CREDIT',StringType(),True) ,
       StructField('INSTALLMENT-AMT',StringType(),True), 
       StructField('CURRENT-BAL',StringType(),True), 
       StructField('OVERDUE-AMT',StringType(),True),
       StructField('WRITE-OFF-AMT',StringType(),True),
       StructField('ASSET_CLASS',StringType(),True),
       StructField('REPORTED DATE - HIST',StringType(),True), 
       StructField('DPD - HIST',StringType(),True), 
       StructField('CUR BAL - HIST',StringType(),True), 
       StructField('AMT OVERDUE - HIST',StringType(),True), 
       StructField('AMT PAID - HIST',StringType(),True),
       StructField('TENURE',StringType(),True),
       StructField('ts',StringType(),True)
       ]))

In [12]:
#Transforming the bureau data 
df_bureau = df_bureau.select(F.from_json(F.col('value').cast('string'), schema_b).alias('parsed_value'))

In [13]:
df_bureau = df_bureau.select(F.explode(F.col('parsed_value')).alias('unnested_value'))

In [14]:
df_bureau = df_bureau.select(
                    F.col("unnested_value.ID").alias("ID"),
                    F.col("unnested_value.SELF-INDICATOR").alias("SELF-INDICATOR"),
                    F.col("unnested_value.MATCH-TYPE").alias("MATCH-TYPE"),
                    F.col("unnested_value.ACCT-TYPE").alias("ACCT-TYPE"),
                    F.col("unnested_value.CONTRIBUTOR-TYPE").alias("CONTRIBUTOR-TYPE"),
                    F.col("unnested_value.DATE-REPORTED").alias("DATE-REPORTED"),
                    F.col("unnested_value.OWNERSHIP-IND").alias("OWNERSHIP-IND"),
                    F.col("unnested_value.ACCOUNT-STATUS").alias("ACCOUNT-STATUS"),
                    F.col("unnested_value.DISBURSED-DT").alias("DISBURSED-DT"),
                    F.col("unnested_value.CLOSE-DT").alias("CLOSE-DT"),
                    F.col("unnested_value.LAST-PAYMENT-DATE").alias("LAST-PAYMENT-DATE"),
                    F.col("unnested_value.CREDIT-LIMIT/SANC AMT").alias("CREDIT-LIMIT/SANC AMT"),
                    F.col("unnested_value.DISBURSED-AMT/HIGH CREDIT").alias("DISBURSED-AMT/HIGH CREDIT"),
                    F.col("unnested_value.INSTALLMENT-AMT").alias("INSTALLMENT-AMT"),
                    F.col("unnested_value.CURRENT-BAL").alias("CURRENT-BAL"),
                    F.col("unnested_value.OVERDUE-AMT").alias("OVERDUE-AMT"),
                    F.col("unnested_value.WRITE-OFF-AMT").alias("WRITE-OFF-AMT"),
                    F.col("unnested_value.ASSET_CLASS").alias("ASSET_CLASS"),
                    F.col("unnested_value.REPORTED DATE - HIST").alias("REPORTED DATE - HIST"),
                    F.col("unnested_value.DPD - HIST").alias("DPD - HIST"),
                    F.col("unnested_value.CUR BAL - HIST").alias("CUR BAL - HIST"),
                    F.col("unnested_value.AMT OVERDUE - HIST").alias("AMT OVERDUE - HIST"),
                    F.col("unnested_value.AMT PAID - HIST").alias("AMT PAID - HIST"),
                    F.col("unnested_value.TENURE").alias("TENURE"),
                    F.col("unnested_value.ts").alias("ts")                    
                )

In [15]:
#Replacing the ',' with '' in numeric columns before typecasting
from pyspark.sql.functions import udf
UDF = udf(lambda s: s.replace(',', '') if s else 'None', StringType())

In [16]:
#Type casting the chosen columns from String Type to appropriate Type

from pyspark.sql.functions import col,when
df_bureau = df_bureau.withColumn("DISBURSED-AMT/HIGH CREDIT",UDF(col("DISBURSED-AMT/HIGH CREDIT")))
df_bureau = df_bureau.withColumn("DISBURSED-AMT/HIGH CREDIT", col("DISBURSED-AMT/HIGH CREDIT").cast(IntegerType()))

df_bureau = df_bureau.withColumn("CURRENT-BAL",UDF(col("CURRENT-BAL")))
df_bureau = df_bureau.withColumn("CURRENT-BAL", col("CURRENT-BAL").cast(IntegerType()))

df_bureau = df_bureau.withColumn("CREDIT-LIMIT/SANC AMT",UDF(col("CREDIT-LIMIT/SANC AMT")))
df_bureau = df_bureau.withColumn("CREDIT-LIMIT/SANC AMT", col("CREDIT-LIMIT/SANC AMT").cast(IntegerType()))

df_bureau = df_bureau.withColumn("OVERDUE-AMT",UDF(col("OVERDUE-AMT")))
df_bureau = df_bureau.withColumn("OVERDUE-AMT", col("OVERDUE-AMT").cast(IntegerType()))

df_bureau = df_bureau.withColumn("CUR BAL - HIST",UDF(col("CUR BAL - HIST")))
df_bureau = df_bureau.withColumn("CUR BAL - HIST", col("CUR BAL - HIST").cast(IntegerType()))

df_bureau = df_bureau.withColumn("WRITE-OFF-AMT",UDF(col("WRITE-OFF-AMT")))
df_bureau = df_bureau.withColumn("WRITE-OFF-AMT", col("WRITE-OFF-AMT").cast(IntegerType()))


df_bureau = df_bureau.withColumn("ID", col("ID").cast(IntegerType()))


df_bureau = df_bureau.withColumn("SELF-INDICATOR", col("SELF-INDICATOR").cast(BooleanType()))

df_bureau = df_bureau.withColumn("TENURE",UDF(col("TENURE")))
df_bureau = df_bureau.withColumn("TENURE", col("TENURE").cast(IntegerType()))

In [17]:
df_bureau = df_bureau.withColumn("AMT PAID - HIST",UDF(col("AMT PAID - HIST")))
df_bureau = df_bureau.withColumn("AMT PAID - HIST", col("AMT PAID - HIST").cast(IntegerType()))

df_bureau = df_bureau.withColumn("AMT OVERDUE - HIST",UDF(col("AMT OVERDUE - HIST")))
df_bureau = df_bureau.withColumn("AMT OVERDUE - HIST", col("AMT OVERDUE - HIST").cast(IntegerType()))
df_bureau = df_bureau.withColumn("ts",col("ts").cast(IntegerType()))

In [18]:
df_bureau = df_bureau.withColumn("ts", from_unixtime('ts').alias('ts'))
df_bureau = df_bureau.withColumn("ts",col("ts").cast(TimestampType()))

In [19]:
#Adding water mark
df_customer = df_customer \
    .withWatermark("ts", "5 seconds")
df_bureau = df_bureau \
    .withWatermark("ts", "5 seconds")

In [20]:
##Q2.4

#Defining a UDF to convert Top-up Month to numeric
def rep_self(string):
    if string == 'false':
        return 0
    else:
        return 1

    #Creating the UDF function
UDF = udf(lambda z: rep_self(z),IntegerType())

In [21]:
df_bureau = df_bureau.withColumn('SELF-INDICATOR', UDF('SELF-INDICATOR'))

In [22]:
#Grouping the data
df_bureau1 = df_bureau \
     .groupBy(df_bureau.ID,F.window(df_bureau.ts, "30 seconds"))\
     .agg(F.sum('WRITE-OFF-AMT').alias('WRITE-OFF-AMT_sum'),
       F.sum('CURRENT-BAL').alias('CURRENT-BAL_sum'),
       F.sum('DISBURSED-AMT/HIGH CREDIT').alias('DISBURSED-AMT/HIGH CREDIT_sum'),
       F.sum('SELF-INDICATOR').alias('SELF-INDICATOR_sum'),
      approx_count_distinct('MATCH-TYPE').alias('MATCH-TYPE_dist'),
      approx_count_distinct('ACCT-TYPE').alias('ACCT-TYPE_dist'),
      approx_count_distinct('CONTRIBUTOR-TYPE').alias('CONTRIBUTOR-TYPE_dist'),
      approx_count_distinct('OWNERSHIP-IND').alias('OWNERSHIP-IND_dist'),
      approx_count_distinct('ACCOUNT-STATUS').alias('ACCOUNT-STATUS_dist'),
      approx_count_distinct('DISBURSED-DT').alias('DISBURSED-DT_dist'),
      approx_count_distinct('CLOSE-DT').alias('CLOSE-DT_dist'),
      approx_count_distinct('LAST-PAYMENT-DATE').alias('LAST-PAYMENT-DATE_dist'),
      sum('CREDIT-LIMIT/SANC AMT').alias('CREDIT-LIMIT/SANC AMT_sum'),

      approx_count_distinct('INSTALLMENT-AMT').alias('INSTALLMENT-AMT_dist'),

      F.sum('OVERDUE-AMT').alias('OVERDUE-AMT_sum'),

      approx_count_distinct('ASSET_CLASS').alias('ASSET_CLASS_dist'),
      approx_count_distinct('REPORTED DATE - HIST').alias('REPORTED DATE - HIST_dist'),
      approx_count_distinct('DPD - HIST').alias('DPD - HIST_dist'),
      F.sum('CUR BAL - HIST').alias('CUR BAL - HIST_sum'),
      F.sum('AMT OVERDUE - HIST').alias('AMT OVERDUE - HIST_dist'),
      F.sum('TENURE').alias('TENURE_sum'),
      F.sum('AMT PAID - HIST').alias('AMT PAID - HIST_dist')
       )

In [23]:
##Q2.5
df_bureau1 = df_bureau1.withColumn('window_start', col("window.start"))

In [24]:
df_bureau1 = df_bureau1.withColumn('window_end', col("window.end"))

In [53]:
#Selecting columns which are used by the provided top-up model
df_bureau2 = df_bureau1.select('ID','WRITE-OFF-AMT_sum','CURRENT-BAL_sum','DISBURSED-AMT/HIGH CREDIT_sum','SELF-INDICATOR_sum','ACCT-TYPE_dist','window_end','window_start')

In [60]:
df_customer1 = df_customer.select('ID',"EMI","MonthlyIncome",'AssetCost','DisbursalAmount',"LoanStatus","BranchID","Area","Tenure","ManufacturerID","City","State",'ts','Top-up Month')

In [61]:
#Joining the 2 streaming dataframes 
from pyspark.sql.functions import col
df_inner_join = df_customer1.join(df_bureau2, on=['ID'], how='inner').where((df_customer1['ts']<=df_bureau2['window_end'])|(df_customer1['ts']>=df_bureau2['window_start']))

In [62]:
##Q2.6
df_inner_join = df_inner_join.withColumn('Top-up_Month',col('Top-up Month'))
df_inner_join1 =  df_inner_join.select('ID', 'window_start', 'window_end', 'ts','Top-up_Month')
df_inner_join = df_inner_join.drop('Top-up Month')

In [30]:
# Write into parquet file
query_file_sink = df_inner_join1.writeStream.format("parquet")\
        .outputMode("append")\
        .option("path", "parquet/df_inner_join1")\
        .option("checkpointLocation", "parquet/df_inner_join1/checkpoint")\
        .start()

In [34]:
query_file_sink.stop()

In [33]:
query_file_sink_df = spark.read.parquet("parquet/df_inner_join1")
query_file_sink_df.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- window_start: timestamp (nullable = true)
 |-- window_end: timestamp (nullable = true)
 |-- ts: timestamp (nullable = true)
 |-- Top-up_Month: string (nullable = true)



In [35]:
##Q2.7

#loading the pipeline model
from pyspark.ml import PipelineModel
pipelineModel = PipelineModel.load('topup_pipeline_model')

In [63]:
#Getting the predictions
predictions = pipelineModel.transform(df_inner_join)

In [64]:
#Selecting columns to store into parquet
df_inner_join2 = predictions.select('ID', 'window_start', 'window_end', 'ts','Top-up_Month','prediction')

In [65]:
df_inner_join2 = df_inner_join2.withColumn('Top-up_Month',col('Top-up_Month'))

In [39]:
# Write into parquet file
query_file_sink1 = df_inner_join2.writeStream.format("parquet")\
        .outputMode("append")\
        .option("path", "parquet/df_11")\
        .option("checkpointLocation", "parquet/df_11/checkpoint")\
        .start()

In [40]:
query_file_sink1.stop()

In [46]:
query_file_sink_df1 = spark.read.parquet("parquet/df_11")
query_file_sink_df1.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- window_start: timestamp (nullable = true)
 |-- window_end: timestamp (nullable = true)
 |-- ts: timestamp (nullable = true)
 |-- Top-up_Month: string (nullable = true)
 |-- prediction: double (nullable = false)



In [66]:
#Schema for parquet
schem = StructType([
       StructField('S',StringType(),True),
])

In [67]:
#Selecting necessary columns
df11 = predictions.select("window_end","State",'prediction')

In [51]:
##Question 2.8

import pandas
import json
def foreach_batch_function(df, epoch_id):
    #For non-empty dataframe
    if df.count()>0:
        
        #Perform aggregation
        df1 = df \
        .select("window_end","State",'prediction')\
        .groupBy('State','window_end')\
        .agg(F.sum('prediction').alias("count"))\
        .select('window_end',F.to_json(F.struct('State','count')))
        
        #renaming columns
        df1 = df1.withColumn('key',col('window_end'))
        df1 = df1.withColumn('value',col("to_json(struct(State, count))"))
        df1 = df1.drop('window_end',"to_json(struct(State, count))")
        df2 = df1.toPandas()
        
        #Converting to key value pair json 
        df_new = df2.set_index('key')['value'].to_json()
        print("epoch_id:",epoch_id)
        print("Count:",df.count())
        print(df_new)
        return df_new
    else:
        pass

In [84]:
#Accessing the foreachbatch function and saving output to parquet
new_df = df11.writeStream.format("parquet").outputMode("append")\
        .foreachBatch(foreach_batch_function)\
        .option("path", "parquet/query1")\
        .option("checkpointLocation", "parquet/query1/checkpoint")\
        .start()

In [80]:
new_df.stop()

In [85]:
#Reading from parquet
query_read = spark.readStream.schema(schem).parquet("parquet/query1")

In [75]:
#Writing to kafka
query_read\
    .writeStream \
    .format("kafka")\
    .option("kafka.bootstrap.servers", "localhost:9092")\
    .option("topic", "stream_data")\
    .option("checkpointLocation", "ckpt/demo")\
    .start()

epoch_id: 7
Count: 121
{"1666074810000":"{\"State\":\"MADHYA PRADESH\",\"count\":26.0}","1666074780000":"{\"State\":\"MADHYA PRADESH\",\"count\":6.0}"}
epoch_id: 8
Count: 50
{"1666074840000":"{\"State\":\"MADHYA PRADESH\",\"count\":15.0}"}
epoch_id: 9
Count: 63
{"1666074870000":"{\"State\":\"MADHYA PRADESH\",\"count\":19.0}"}
epoch_id: 10
Count: 65
{"1666074900000":"{\"State\":\"MADHYA PRADESH\",\"count\":24.0}"}
epoch_id: 11
Count: 104
{"1666074960000":"{\"State\":\"MADHYA PRADESH\",\"count\":20.0}","1666074930000":"{\"State\":\"MADHYA PRADESH\",\"count\":20.0}"}
epoch_id: 12
Count: 38
{"1666074990000":"{\"State\":\"MADHYA PRADESH\",\"count\":19.0}"}
epoch_id: 13
Count: 121
{"1666075050000":"{\"State\":\"MADHYA PRADESH\",\"count\":17.0}","1666075020000":"{\"State\":\"MADHYA PRADESH\",\"count\":27.0}"}
epoch_id: 14
Count: 46
{"1666075080000":"{\"State\":\"MADHYA PRADESH\",\"count\":12.0}"}
epoch_id: 15
Count: 58
{"1666075110000":"{\"State\":\"MADHYA PRADESH\",\"count\":22.0}"}
epoch_id